## Build a water quality model

This notebook demonstrates how to prepare **DELWAQ D-Water Quality** model from scratch using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### hydroMT CLI build interface

In hydroMT, you can interact with **DELWAQ models** either to be used by **D-Emissions (EM)** or **D-Water Quality (WQ)**.

Lets first check if the delwaq model is recognized by hydromt

In [ ]:
# this should return "delwaq, wflow, wflow_sediment"
!hydromt --models

Using the **hydromt build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [ ]:
!hydromt build --help

### Setup delwaq EM model base layers 

DELWAQ models are quite specific in the sense that they are usually added on top of an already existing hydrologic / hydraulic or hydrodynamic model.
In hydroMT, for now, DELWAQ models (EM for D-Emissions and WQ for D-Water Quality) can only be built on top of [Wflow](https://deltares.github.io/Wflow.jl/stable/) hydrologic models. You can find more information on this coupling in [docs(coupling_wflow)](https://deltares.github.io/hydromt_delwaq/latest/advanced/coupling_wflow.html).

Here is how you can build a DELWAQ model:

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build delwaq "./WQ_test_base" -r "{'wflow': 'wflow_piave'}" --opt global.mtype=WQ -vvv

The example above means the following: run **hydromt build** with:

* `delwaq` : i.e. build a delwaq model
* `./WQ_test_base` : output model folder
* `-r or --region "{'wflow': 'wflow_piave'}"` : derive the model from the existing wflow model located in wflow_piave folder. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options). For delwaq, the only possible options is from an existing wflow model.
* `-opt global.mtype=WQ` : specifies which type of delwaq model to build. Here **WQ** for D-Water Quality.
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

NOTE: As we did not specify a model configuration, only the base maps (grid, segment IDs) have been setup using default parameters. To build a complete model we need the use a configuraton ini-file.

### Model setup configuration

The ini-file contains the model setup configuration and determines which components are build and in which order and optionally sets non-default arguments for each component. This configuration is passed to hydromt using `-i <path_to_ini_file>`. We have prepared several example ini-files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_delwaq/tree/main/examples) and from the [docs(build_configuration)](https://deltares.github.io/hydromt_delwaq/latest/user_guide/build_configuration.html). 

Each header as shown between `[...]` (e.g. `[setup_basemaps]`) corresponds to a model component. All model components are explained in the [docs(model_components)](https://deltares.github.io/hydromt_delwaq/latest/user_guide/components.html). 

**Almost each DELWAQ model is unique** depending on which substances and sources are included but also what kind of emission data is available. For these reasons, there is no default build ini file for a delwaq model and still some manual steps required to build and run a D-Water Quality model with hydroMT. You can learn more about these steps in the [docs(hydromt_D-Water Quality)](https://deltares.github.io/hydromt_delwaq/latest/advanced/generic_delwaq.html#water-quality-modelling-with-d-water-quality).

We will load an example delwaq build WQ ini file for inspection:

In [ ]:
fn_ini = 'delwaq_build_WQ.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

Some explanations for the sections in the ini file above:

* **global**: options true for every model components of delwaq. This is where you can specify if you want to build an **EM** or **WQ** model using the *mtype* option. You can also add local data libaries in *data_libs* instead f in the command line.
* **setup_basemaps**: prepares the first delwaq model layers (grid, segment ID) and for WQ also the pointer.
* **setup_monitoring**: prepares monitoring points using the gauges locations of the wflow model and the monitoring areas by compartments (surface waters only here)
* **setup_hydrology_forcing**: prepares hydrological data from wflow outputs. Information on the *wflow_output* source is available in the *local_sources.yml*.

For more information on all the options used (resampling method, NaN values handling...), please check the [docs(model_components)](https://deltares.github.io/hydromt_delwaq/latest/user_guide/components.html)

### Data for model setup

The `<...>_fn` arguments correspond to a data source from the `DataCatalog` which is based on a yml-file with references to the data paths/urls and how the data should be read. This file can be provided to hydromt using `-d <path_to_yml_file>`. 

By default some example data for the Piave basin will be downloaded to `~/.hydromt_data/` which is also used for this example. An overview of the available example data is provided [here](https://deltares.github.io/hydromt/latest/user_guide/data.html#available-global-datasets)
This example data is a based on the data which are available from the Deltares p-drive. 
If you have acces to this drive, a pre-configured catalog file can be loaded using the `--dd` flag. 

More background how to write a data catalog yml file can be found in the [hydromt core docs](https://deltares.github.io/hydromt/latest/user_guide/data.html) 

### Setup complete delwaq WQ model 

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build delwaq "./WQ_test_full" -r "{'wflow': 'wflow_piave'}" -i delwaq_build_WQ.ini -d artifact_data -vv

With this example we build a complete delwaq WQ model including hydrological data. Compared to the previous **hydromt build** we have added:
* `-i delwaq_build_WQ.ini` : setup configuration file including all components to build and their arguments

Note that the `--opt global.mtype=WQ` is not present anymore, as this option is directly set in the delwaq_build_WQ.ini file.

Next we check which files and folders have been created:

* **config**: basic configuration setups for the Delwaq input file .inp. Names of the ASCII files start with the corresponding block in the .inp file.
* **dynamicdata**: Delwaq time-dependant data (hydrological fluxes) in binary format.
* **fews**: additional files for connection with Delft-FEWS (not implemented yet)
* **hydromodel**: data extracted to connect the hydrological model (wflow) and Delwaq (tif format). These files are not mandatory for Delwaq but produced extra from HydroMT.
* **staticdata**: static (spatial but non time-dependant) data for Delwaq (typically emission data) in binary format. A copy of the data in NetCDF format is also available for easier visualization.
* **staticgeoms**: related geometry files (geojson format). These files are not mandatory for Delwaq but produced extra from HydroMT.

In [ ]:
import os
root = 'WQ_test_full'
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith('.xml'):
            continue
        print(f' - {name}')

You can see that we didn't create any emission data here. In some cases, you may require some additional GIS data in order to run a WQ model, for example, when you do not need to run the D-Emissions model beforehand, but include point sources that emit pollutants into the surface water directly. When this is the case, you can still use the various [setup_emission_*] components of hydroMT_delwaq.

### Visualize and/or inspect model schematization

* The **delwaq plot** example notebook contains scripts to visualize your model (staticdata, dynamicdata, hydromodel and staticgeoms).

In the meantime, feel free to have a look at some configuration files that were prepared by HydroMT. You can change the file name from the example code below.

In [ ]:
import os
model_path = './WQ_test_full'
fn_config = 'config/B3_nrofseg.inc'
with open(os.path.join(model_path,fn_config), 'r') as f:
    txt = f.read()
print(txt)

In the config folder, you can also see two pointer files:

* B4_pointer.poi (Binary version)
* B4_pointer.inc (ASCII version)

You can also see that a NetCDF file was created: **B3_waqgeom.nc**. This file can be used to produce NetCDF outputs directly when running D-Emissions or D-Water Quality but also to visualize the model in FEWS.